<a href="https://colab.research.google.com/github/CSSamarasinghe/SE4050_Assignment/blob/IT21263194/After%20data%20set%20change%20imporvement%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'amazon-reviews-for-sentianalysis-finegrained-csv:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2078107%2F3499094%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241003%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241003T164549Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D25a30a2ad799910d48f45f67793fb1d09f31560ae3773fa14870108018fa213261ae62aeb020867952a30d8094a21359c2546a6475193ac986bc2645a191d848c7223dbd5c68600aeb8f17d5a979be9750827328816b6743cf0d74ca79e5a28878f8606f4d3cd2176678d18b8a70a20bbf9be8037aa238d4b56bd1a0ba1a8d1fb4003bbd12f51fa6d438aaa8a7e8fa90a9291e62969f7df78c34dc6a71c76fb2f1c11e2964127136ec4d5554e9f1bdd7c530f8ddf987a08dd6cb91a849e4e98a2089149d22cb00ca687688c756071af4270e392fd79699c1f4a35ca16597523ca8bfe37fea11d5911465ea51d6eb17c1e3663da806763c8ef4e96e1f0a9ae9ab'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 654512809 bytes downloaded
Downloaded and uncompressed: amazon-reviews-for-sentianalysis-finegrained-csv
Data source import complete.


In [5]:
import pandas as pd

# Load the dataset
train = pd.read_csv('../input/amazon-reviews-for-sentianalysis-finegrained-csv/amazon_review_fine-grained_5_classes_csv/train.csv')
test = pd.read_csv('../input/amazon-reviews-for-sentianalysis-finegrained-csv/amazon_review_fine-grained_5_classes_csv/test.csv')

# Check the column names
print("Training data set")
print(train.columns)
print(train.shape)

print("Testing data set")
print(test.columns)
print(test.shape)

Training data set
Index(['class_index', 'review_title', 'review_text'], dtype='object')
(3000000, 3)
Testing data set
Index(['class_index', 'review_title', 'review_text'], dtype='object')
(650000, 3)


In [6]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tensorflow.keras.preprocessing.text import Tokenizer
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import OneHotEncoder

# Load the dataset
train_data = pd.read_csv('../input/amazon-reviews-for-sentianalysis-finegrained-csv/amazon_review_fine-grained_5_classes_csv/train.csv')
test_data = pd.read_csv('../input/amazon-reviews-for-sentianalysis-finegrained-csv/amazon_review_fine-grained_5_classes_csv/test.csv')

# Reduce the dataset to a manageable size (e.g., taking the first 1000 samples)
train_data = train_data.sample(n=60000, random_state=42)  # Randomly sample 1000 rows from the training data
test_data = test_data.sample(n=25000, random_state=42)      # Randomly sample 200 rows from the test data

# Initialize the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['review_text'])

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(train_data['review_text'])
X_test = tokenizer.texts_to_sequences(test_data['review_text'])

# Directly take the numeric values from class_index
Y_train = train_data['class_index'].values
Y_test = test_data['class_index'].values

# If needed, one-hot encode the labels for multi-class classification
Y_train_reshaped = Y_train.reshape(-1, 1)
Y_test_reshaped = Y_test.reshape(-1, 1)

# Use sparse_output instead of sparse
encoder = OneHotEncoder(sparse_output=False)
Y_train_onehot = encoder.fit_transform(Y_train_reshaped)
Y_test_onehot = encoder.transform(Y_test_reshaped)

# Print the shapes to confirm
print(f'X_train shape: {len(X_train)}, Y_train shape: {Y_train_onehot.shape}')
print(f'X_test shape: {len(X_test)}, Y_test shape: {Y_test_onehot.shape}')

# Pad the sequences and convert to tensors
X_train_padded = pad_sequence([torch.tensor(seq) for seq in X_train], batch_first=True, padding_value=0)
X_test_padded = pad_sequence([torch.tensor(seq) for seq in X_test], batch_first=True, padding_value=0)

# Adjust the input size
input_size = len(tokenizer.word_index) + 1  # Vocabulary size (plus 1 for padding)

# Create TensorDataset and DataLoader
train_dataset = TensorDataset(X_train_padded, torch.tensor(Y_train_onehot, dtype=torch.float32))
test_dataset = TensorDataset(X_test_padded, torch.tensor(Y_test_onehot, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

X_train shape: 60000, Y_train shape: (60000, 5)
X_test shape: 25000, Y_test shape: (25000, 5)


In [13]:
!wget http://nlp.stanford.edu/data/glove.6B.zip



--2024-10-05 00:12:39--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-10-05 00:12:39--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-10-05 00:12:39--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.5’

gl

In [16]:
!unzip glove.6B.zip


Archive:  glove.6B.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of glove.6B.zip or
        glove.6B.zip.zip, and cannot find glove.6B.zip.ZIP, period.


In [17]:
!ls

glove.6B.zip	glove.6B.zip.2	glove.6B.zip.4	sample_data
glove.6B.zip.1	glove.6B.zip.3	glove.6B.zip.5


In [18]:
import requests
import zipfile
import os

def download_glove(glove_url, output_dir):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Download the GloVe file
    response = requests.get(glove_url)
    zip_file_path = os.path.join(output_dir, 'glove.6B.zip')

    with open(zip_file_path, 'wb') as f:
        f.write(response.content)

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)

    print(f"GloVe embeddings downloaded and extracted to {output_dir}")

# URL for GloVe 300-dimensional embeddings
glove_url = "http://nlp.stanford.edu/data/glove.6B.zip"
output_directory = "./glove_embeddings"

download_glove(glove_url, output_directory)

GloVe embeddings downloaded and extracted to ./glove_embeddings


In [20]:
import numpy as np

def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.array(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Load GloVe embeddings
glove_file_path = './glove_embeddings/glove.6B.300d.txt'  # Update this path if necessary
glove_embeddings = load_glove_embeddings(glove_file_path)
print(f'Loaded {len(glove_embeddings)} word vectors.')

# Function to print embeddings for specific words
def print_word_embeddings(words, embeddings):
    for word in words:
        if word in embeddings:
            print(f"Embedding for '{word}': {embeddings[word]}")
        else:
            print(f"'{word}' not found in GloVe model.")

# Specify words to print
words_to_print = ['hello', 'world', 'example']  # Add any words you want to check
print_word_embeddings(words_to_print, glove_embeddings)

Loaded 400000 word vectors.
Embedding for 'hello': [-3.3712e-01 -2.1691e-01 -6.6365e-03 -4.1625e-01 -1.2555e+00 -2.8466e-02
 -7.2195e-01 -5.2887e-01  7.2085e-03  3.1997e-01  2.9425e-02 -1.3236e-02
  4.3511e-01  2.5716e-01  3.8995e-01 -1.1968e-01  1.5035e-01  4.4762e-01
  2.8407e-01  4.9339e-01  6.2826e-01  2.2888e-01 -4.0385e-01  2.7364e-02
  7.3679e-03  1.3995e-01  2.3346e-01  6.8122e-02  4.8422e-01 -1.9578e-02
 -5.4751e-01 -5.4983e-01 -3.4091e-02  8.0017e-03 -4.3065e-01 -1.8969e-02
 -8.5670e-02 -8.1123e-01 -2.1080e-01  3.7784e-01 -3.5046e-01  1.3684e-01
 -5.5661e-01  1.6835e-01 -2.2952e-01 -1.6184e-01  6.7345e-01 -4.6597e-01
 -3.1834e-02 -2.6037e-01 -1.7797e-01  1.9436e-02  1.0727e-01  6.6534e-01
 -3.4836e-01  4.7833e-02  1.6440e-01  1.4088e-01  1.9204e-01 -3.5009e-01
  2.6236e-01  1.7626e-01 -3.1367e-01  1.1709e-01  2.0378e-01  6.1775e-01
  4.9075e-01 -7.5210e-02 -1.1815e-01  1.8685e-01  4.0679e-01  2.8319e-01
 -1.6290e-01  3.8388e-02  4.3794e-01  8.8224e-02  5.9046e-01 -5.3515e-02


In [21]:
# Prepare embedding matrix
embedding_dim = 300  # Dimension of GloVe vectors
embedding_matrix = np.zeros((input_size, embedding_dim))

for word, i in tokenizer.word_index.items():
    if i < input_size:  # Ensure we don't exceed the matrix size
        embedding_vector = glove_embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size, embedding_matrix, dropout_rate=0.5):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))  # Load pre-trained weights

        self.lstm1 = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)

        self.layer_norm1 = nn.LayerNorm(hidden_size)  # Layer normalization after lstm1
        self.layer_norm2 = nn.LayerNorm(hidden_size)  # Layer normalization after lstm2

        self.dropout = nn.Dropout(dropout_rate)  # Dropout layer
        self.fc1 = nn.Linear(hidden_size, hidden_size)  # Additional fully connected layer
        self.fc2 = nn.Linear(hidden_size, output_size)  # Output layer

        # Leaky ReLU with a negative slope of 0.01
        self.leaky_relu = nn.LeakyReLU(0.01)

    def forward(self, x):
        x = self.embedding(x)

        x, _ = self.lstm1(x)
        x = self.layer_norm1(x)  # Apply layer normalization after the first LSTM
        x = self.leaky_relu(x)    # Use Leaky ReLU activation after lstm1

        x, _ = self.lstm2(x)
        x = self.layer_norm2(x)  # Apply layer normalization after the second LSTM

        x = self.dropout(x[:, -1, :])  # Apply dropout to the last time step output
        x = self.fc1(x)                 # First fully connected layer
        x = F.relu(x)                   # ReLU activation function for fully connected layer
        x = self.fc2(x)                 # Output layer

        return x

In [25]:
import numpy as np
import torch
import torch.nn as nn

# Assuming you have already defined and loaded your GloVe embeddings and created the embedding matrix
# Load GloVe embeddings and create embedding_matrix (as discussed previously)

# Set hyperparameters
num_epochs = 5  # Number of training epochs
embedding_dim = 300  # Dimension of GloVe embeddings (adjust based on your GloVe file)
hidden_size = 64  # Number of LSTM hidden units
output_size = len(set(Y_train))  # Number of classes based on your training labels
learning_rate = 0.001  # Learning rate for the optimizer

# Ensure input_size is defined based on your tokenizer
input_size = len(tokenizer.word_index) + 1  # Vocabulary size (plus padding)

# Initialize the model **before** defining the optimizer
model = LSTMModel(vocab_size=input_size,
                  embedding_dim=embedding_dim,
                  hidden_size=hidden_size,
                  output_size=output_size,
                  embedding_matrix=embedding_matrix,  # Ensure this is defined
                  dropout_rate=0.5)  # You can adjust the dropout rate as needed

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for multi-class classification
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
import torch.optim as optim
from torch.utils.data import DataLoader

# Loss and optimizer
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    epoch_loss = 0  # Initialize loss for the epoch

    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Clear the gradients

        inputs = inputs.long()  # Ensure inputs are long type for embedding

        outputs = model(inputs)  # Forward pass

        # Compute loss (labels should be LongTensor of class indices)
        loss = criterion(outputs, labels.argmax(dim=1))  # Use argmax to get class indices from one-hot labels

        # Backward pass and optimization
        loss.backward()  # Backward pass
        optimizer.step()  # Optimization step

        epoch_loss += loss.item()  # Accumulate loss for this batch

    avg_loss = epoch_loss / len(train_loader)  # Calculate average loss for the epoch
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}')

Epoch [1/5], Loss: 1.6128
Epoch [2/5], Loss: 1.6100
Epoch [3/5], Loss: 1.6099
Epoch [4/5], Loss: 1.6096
Epoch [5/5], Loss: 1.6096


In [27]:
import torch
from torch.utils.data import DataLoader

# Evaluation function
def evaluate_model(model, data_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    # Disable gradient calculation for evaluation
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.long()  # Ensure inputs are long type for embedding
            inputs, labels = inputs.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')

            # Forward pass through the model
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Get predicted class indices
            _, predicted_classes = torch.max(outputs.data, 1)

            # Convert one-hot encoded labels back to class indices for comparison
            true_classes = torch.argmax(labels, dim=1)

            total_samples += labels.size(0)
            correct_predictions += (predicted_classes == true_classes).sum().item()

    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions / total_samples * 100.0

    return avg_loss, accuracy

# Example usage: Evaluate on the test set
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

Test Loss: 1.6095, Test Accuracy: 19.85%
